<a href="https://colab.research.google.com/github/ozawa-nami/test/blob/main/%E3%80%90%E5%85%B1%E6%9C%89%E7%94%A8%E3%80%91%E7%AB%B6%E5%90%88%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
from google.auth import default
import gspread

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)



!pip install selenium
!pip install webdriver_manager pyvirtualdisplay
!apt-get update
!apt-get install -y xvfb wget
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt-get install -y ./google-chrome-stable_current_amd64.deb

from pyvirtualdisplay import Display
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 仮想ディスプレイの起動
display = Display(visible=0, size=(1024, 768))
display.start()

# Chrome のオプションとサービスの設定
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--headless')

s = Service(ChromeDriverManager().install())

# Chrome ブラウザの起動
driver = webdriver.Chrome(service=s, options=options)


In [ ]:
#自分でURL入れる

# 必要なライブラリのインストール
!pip install gspread beautifulsoup4 requests

import gspread
import google.auth
from google.colab import auth
from bs4 import BeautifulSoup
import requests
from urllib.parse import urlparse, parse_qs
import time
import re
from oauth2client.client import GoogleCredentials
import chardet
import json  # JSONデータの処理用

# Googleドライブの認証
auth.authenticate_user()
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

#★ スプレッドシートのURLとシート名
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1HyK034F_9oTnRlPpRsWMHNpXILbnuR52MWn2Ueh5Lpw/edit#gid=1960992532"
worksheet_name = "{エンジニアネガポジ}競合分析"

# スプレッドシートの読み込み
sh = gc.open_by_url(spreadsheet_url)
worksheet = sh.worksheet(worksheet_name)

# A列からURLを取得（例として5行目から20行目までのURLを取得）
urls = [worksheet.cell(i, 1).value for i in range(5, 25)]



# classify_link関数の定義
def classify_link(href, base_url):
    parsed_url = urlparse(href)
    if parsed_url.netloc == base_url:
        return "内部リンク"
    elif parsed_url.netloc:
        return "外部リンク"
    elif parsed_url.fragment:
        return "ページ内リンク"
    else:
        return "不明なリンク"

# A1セルからキーワードを取得
keywords_string = worksheet.cell(1, 1).value
# キーワードをカンマで区切ってリストに変換
keywords_list = [keyword.strip() for keyword in keywords_string.split(',')]


# 各URLに対してスクレイピングと情報取得
for i, url in enumerate(urls, start=5):
    if not url:
        for j in range(2, 12):  # B列からL列まで
            worksheet.update_cell(i, j, "")  # 空文字を書き込む
        continue

    # リクエストの送信
    try:
        res = requests.get(url)
        res.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request error for {url}: {e}")
        for j in range(2, 12):  # B列からL列まで
            worksheet.update_cell(i, j, "")  # 空文字を書き込む
        continue

    # ページの解析
    soup = BeautifulSoup(res.text, 'html.parser')  # この行をforループ内に移動

    # 文字コードを自動判別
    detected_encoding = chardet.detect(res.content)['encoding']
    # ページの解析
    soup = BeautifulSoup(res.content, 'html.parser', from_encoding=detected_encoding)

    # サイトのテキストを取得
    site_text = soup.get_text()

    # H1タイトル
    h1_tag = soup.find('h1')
    h1_title = h1_tag.text.strip() if h1_tag else "N/A"

    # H2とH3の見出しの取得
    headers = soup.find_all(['h2', 'h3'])
    headers_list = []
    for h in headers:
        if h.name == 'h2':
            headers_list.append(f"H2: {h.text.strip()}")
        elif h.name == 'h3':
            headers_list.append(f"H3: {h.text.strip()}")
    headers_text = "\n".join(headers_list)

    # サイトの文字数
    site_text_length = len(re.sub(r'\s+', ' ', site_text))

    # ディスクリプション（metaタグから取得）
    description = soup.find('meta', attrs={'name': 'description'})
    description_content = description.get('content').strip() if description else "N/A"

    # リンク数のカウント
    links = soup.find_all('a')
    link_count = len(links)



  # classify_link関数の定義

    def classify_link(href, base_url):
        parsed_url = urlparse(href)
        if parsed_url.netloc == base_url:
            return "内部リンク"
        elif parsed_url.netloc and base_url not in parsed_url.netloc:
            return "外部リンク"
        elif parsed_url.path.startswith("/"):
            return "内部リンク"
        elif parsed_url.fragment:
            return "ページ内リンク"
        else:
            return "不明なリンク"


    # 発リンクおよび内部リンクのカウント
    base_url = urlparse(url).netloc
    internal_links = []
    external_links = []
    article_links = []

    for link in links:
        href = link.get('href')
        if href:
            link_type = classify_link(href, base_url)
            if link_type == "内部リンク":
                internal_links.append((href, link.text))
            elif link_type == "外部リンク":
                external_links.append((href, link.text))
            elif link_type == "ページ内リンク":
                article_links.append((href, link.text))

    # 外部リンクの整形
    external_links_count = len(external_links)
    external_links_text = "\n".join([f"{text}: {href}" for href, text in external_links])

    # 内部リンクの整形
    internal_links_count = len(internal_links)
    internal_links_text = "\n".join([f"{text}: {href}" for href, text in internal_links])

    # 記事内リンクの整形
    article_links_count = len(article_links)
    article_links_text = "\n".join([f"{text}: {href}" for href, text in article_links])


    # 画像リンクとその数の取得
    images = soup.find_all('img')
    image_count = len(images)
    image_links = [img.get('src') for img in images if img.get('src')]
    image_links_formatted = "\n".join(image_links)

    # 画像リンクのテキストを切り詰める
    truncated_image_links_formatted = image_links_formatted[:50000]


    # JSONデータを抽出
    soup = BeautifulSoup(res.content, 'html.parser')
    json_ld_scripts = soup.find_all('script', {'type': 'application/ld+json'})

    # JSONデータが複数ある場合の処理
    date_published = "N/A"
    date_modified = "N/A"

    for script in json_ld_scripts:
        try:
            json_data = json.loads(script.string)
            if isinstance(json_data, list):
                # JSONデータがリストの場合、各要素をチェック
                for item in json_data:
                    if item.get('@type') in ['BlogPosting', 'Article', 'WebPage']:
                        date_published = item.get('datePublished', "N/A")
                        date_modified = item.get('dateModified', "N/A")
                        break
            elif json_data.get('@type') in ['BlogPosting', 'Article', 'WebPage']:
                date_published = json_data.get('datePublished', "N/A")
                date_modified = json_data.get('dateModified', "N/A")
            elif json_data.get('@graph'):
                # グラフ構造を持つJSONデータの場合
                graph_data = json_data.get('@graph')
                for item in graph_data:
                    if item.get('@type') in ['BlogPosting', 'Article', 'WebPage']:
                        date_published = item.get('datePublished', "N/A")
                        date_modified = item.get('dateModified', "N/A")
                        break
        except json.JSONDecodeError:
            # JSONデータのパースに失敗した場合
            continue
        if date_published != "N/A" and date_modified != "N/A":
            # 有効なデータが見つかった場合
            break

    # date_publishedとdate_modifiedを出力
    print(f"URL: {url}")
    print(f"Date Published: {date_published}")
    print(f"Date Modified: {date_modified}")


    # 各キーワードの出現数をカウント
    keyword_counts = {}
    for keyword in keywords_list:
        keyword_count = site_text.lower().count(keyword.lower())  # 大小文字を無視してカウント
        keyword_counts[keyword] = keyword_count
    keyword_counts_text = "\n".join([f"{k}: {v}" for k, v in keyword_counts.items()])

    # 出力
    worksheet.update_cell(i, 2, h1_title)  # B列にタイトルを書き込む
    worksheet.update_cell(i, 3, headers_text)  # C列にH2H3情報を書き込む
    worksheet.update_cell(i, 4, site_text_length)  # D列にサイトの文字数を書き込む
    worksheet.update_cell(i, 5, description_content)  # E列にサイトのディスクリプションを書き込む
    worksheet.update_cell(i, 6, f"{external_links_count}\n{external_links_text}")  # F列に外部リンク数とリンクを書き込む

    truncated_internal_links_text = internal_links_text[:50000]
    worksheet.update_cell(i, 7, f"{internal_links_count}\n{truncated_internal_links_text}")   # G列に内部リンク数とリンクを書き込む

    # worksheet.update_cell(i, 7, f"{internal_links_count}\n{internal_links_text}")  # G列に内部リンク数とリンクを書き込む

    worksheet.update_cell(i, 8, article_links_count)  # H列にページ内リンク数を書き込む

        # I列に画像数とリンクを書き込む（画像リンクのテキストを切り詰めたバージョンを使用）
    worksheet.update_cell(i, 9, f"{image_count}\n{truncated_image_links_formatted}")

    worksheet.update_cell(i, 10, date_published)  # J列にDate Publishedを書き込む
    worksheet.update_cell(i, 11, date_modified)  # K列にDate Modifiedを書き込む
    worksheet.update_cell(i, 12, keyword_counts_text)  # L列にキーワード出現数を書き込む


    # APIの利用制限を考慮して待ち時間を設ける
    time.sleep(2)


URL: https://www.workport.co.jp/
Date Published: N/A
Date Modified: N/A
URL: https://www.a-tm.co.jp/top/job-change/workport/
Date Published: 2023-03-02T00:00:00+09:00
Date Modified: 2023-10-10T16:46:07+09:00
URL: https://axxis.co.jp/magazine/55349
Date Published: 2022-06-14T16:39:55+09:00
Date Modified: 2023-10-16T21:04:52+09:00
URL: https://www.maneo.jp/career/jc-workport-reputation/
Date Published: 2023-10-03T13:41:36+0900
Date Modified: 2023-10-03T13:41:37+0900
URL: https://www.jesra.or.jp/search/1239/
Date Published: N/A
Date Modified: N/A
URL: https://prtimes.jp/main/html/searchrlp/company_id/39106
Date Published: N/A
Date Modified: N/A
URL: https://www.your-intern.com/tenshoku/content/2744/
Date Published: N/A
Date Modified: N/A
URL: https://m.facebook.com/workport/
Date Published: N/A
Date Modified: N/A
URL: https://m.youtube.com/channel/UCxENWeGPMWlyCkcvs9L7AtA
Date Published: N/A
Date Modified: N/A


TypeError: ignored

In [ ]:
#A1セルに入っているKWの上位⚪︎位を分析

import gspread
import google.auth
from google.colab import auth
from bs4 import BeautifulSoup
import requests
from urllib.parse import urlparse, parse_qs
import time
import re
from oauth2client.client import GoogleCredentials
import chardet

# Googleドライブの認証
auth.authenticate_user()
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

#★ スプレッドシートのURLとシート名
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1HyK034F_9oTnRlPpRsWMHNpXILbnuR52MWn2Ueh5Lpw/edit#gid=558593770"
worksheet_name = "上位記事取得用_01"

# スプレッドシートの読み込み
sh = gc.open_by_url(spreadsheet_url)
worksheet = sh.worksheet(worksheet_name)

# キーワードの取得
keyword = worksheet.acell('A1').value
keywords_list = keyword.split(",")

# Google検索のURLを構築
search_url = f"https://www.google.com/search?q={keyword}&num=10"

# リクエストの送信
try:
    res = requests.get(search_url)
    res.raise_for_status()
except requests.exceptions.RequestException as e:
    print(f"Request error: {e}")
    exit()

# ページの解析
soup = BeautifulSoup(res.text, 'html.parser')

# URLの取得
url_cells = soup.select('.kCrYT > a')
urls = []
for url_cell in url_cells:
    href = url_cell.get('href')
    parsed_url = urlparse(href)
    query_params = parse_qs(parsed_url.query)
    if 'q' in query_params:
        url = query_params['q'][0]
        urls.append(url)

# classify_link関数の定義
def classify_link(href, base_url):
    parsed_url = urlparse(href)
    if parsed_url.netloc == base_url:
        return "内部リンク"
    elif parsed_url.netloc:
        return "外部リンク"
    elif parsed_url.fragment:
        return "ページ内リンク"
    else:
        return "不明なリンク"

# 上位10記事のURLをスプレッドシートに書き込む
for i, url in enumerate(urls, start=5):
    worksheet.update_cell(i, 1, url)

print("URLの取得が完了しました。")

# 各URLに対してスクレイピングと情報取得
for i, url in enumerate(urls, start=5):
    if not url:
        for j in range(2, 11):  # B列からJ列まで
            worksheet.update_cell(i, j, "")  # 空文字を書き込む
        continue

    # リクエストの送信
    try:
        res = requests.get(url)
        res.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request error for {url}: {e}")
        for j in range(2, 11):  # B列からJ列まで
            worksheet.update_cell(i, j, "")  # 空文字を書き込む
        continue

    # 文字コードを自動判別
    detected_encoding = chardet.detect(res.content)['encoding']
    # ページの解析
    soup = BeautifulSoup(res.content, 'html.parser', from_encoding=detected_encoding)

    # サイトのテキストを取得
    site_text = soup.get_text()

    # H1タイトル
    h1_tag = soup.find('h1')
    h1_title = h1_tag.text.strip() if h1_tag else "N/A"

    # H2とH3の見出しの取得
    headers = soup.find_all(['h2', 'h3'])
    headers_list = []
    for h in headers:
        if h.name == 'h2':
            headers_list.append(f"H2: {h.text.strip()}")
        elif h.name == 'h3':
            headers_list.append(f"H3: {h.text.strip()}")
    headers_text = "\n".join(headers_list)

    # サイトの文字数
    site_text_length = len(re.sub(r'\s+', ' ', site_text))

    # ディスクリプション（metaタグから取得）
    description = soup.find('meta', attrs={'name': 'description'})
    description_content = description.get('content').strip() if description else "N/A"

    # リンク数のカウント
    links = soup.find_all('a')
    link_count = len(links)



  # classify_link関数の定義
    def classify_link(href, base_url):
        parsed_url = urlparse(href)
        if parsed_url.netloc == base_url:
            return "内部リンク"
        elif parsed_url.netloc and base_url not in parsed_url.netloc:
            return "外部リンク"
        elif parsed_url.path.startswith("/"):
            return "内部リンク"
        elif parsed_url.fragment:
            return "ページ内リンク"
        else:
            return "不明なリンク"


    # 発リンクおよび内部リンクのカウント
    base_url = urlparse(url).netloc
    internal_links = []
    external_links = []
    article_links = []

    for link in links:
        href = link.get('href')
        if href:
            link_type = classify_link(href, base_url)
            if link_type == "内部リンク":
                internal_links.append((href, link.text))
            elif link_type == "外部リンク":
                external_links.append((href, link.text))
            elif link_type == "ページ内リンク":
                article_links.append((href, link.text))

    # 外部リンクの整形
    external_links_count = len(external_links)
    external_links_text = "\n".join([f"{text}: {href}" for href, text in external_links])

    # 内部リンクの整形
    internal_links_count = len(internal_links)
    internal_links_text = "\n".join([f"{text}: {href}" for href, text in internal_links])

    # 記事内リンクの整形
    article_links_count = len(article_links)
    article_links_text = "\n".join([f"{text}: {href}" for href, text in article_links])

    # 画像リンクとその数の取得
    images = soup.find_all('img')
    image_count = len(images)
    image_links = [img.get('src') for img in images if img.get('src')]
    image_links_formatted = "\n".join(image_links)

    # 各キーワードの出現数をカウント
    keyword_counts = {}
    for keyword in keywords_list:
        keyword_count = site_text.count(keyword)
        keyword_counts[keyword] = keyword_count
    keyword_counts_text = "\n".join([f"{k}: {v}" for k, v in keyword_counts.items()])

    # 出力
    worksheet.update_cell(i, 2, h1_title)  # B列にタイトルを書き込む
    worksheet.update_cell(i, 3, headers_text)  # C列にH2H3情報を書き込む
    worksheet.update_cell(i, 4, site_text_length)  # D列にサイトの文字数を書き込む
    worksheet.update_cell(i, 5, description_content)  # E列にサイトのディスクリプションを書き込む
    worksheet.update_cell(i, 6, f"{external_links_count}\n{external_links_text}")  # F列に外部リンク数とリンクを書き込む

    truncated_internal_links_text = internal_links_text[:50000]
    worksheet.update_cell(i, 7, f"{internal_links_count}\n{truncated_internal_links_text}")

    # worksheet.update_cell(i, 7, f"{internal_links_count}\n{internal_links_text}")  # G列に内部リンク数とリンクを書き込む

    worksheet.update_cell(i, 8, article_links_count)  # H列にページ内リンク数を書き込む
    worksheet.update_cell(i, 9, f"{image_count}\n{image_links_formatted}")  # I列に画像数とリンクを書き込む
    worksheet.update_cell(i, 10, keyword_counts_text)  # J列にキーワード出現数を書き込む

    # APIの利用制限を考慮して待ち時間を設ける
    time.sleep(2)


URLの取得が完了しました。
Request error for https://exidea.co.jp/blog/job/job-change/it-tenshoku-agent/: HTTPSConnectionPool(host='exidea.co.jp', port=443): Max retries exceeded with url: /blog/job/job-change/it-tenshoku-agent/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)')))
Request error for https://mynavi-agent.jp/it/: 403 Client Error: Forbidden for url: https://mynavi-agent.jp/it/
